# Изучение данных 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

In [5]:
con=engine.connect()

In [6]:
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
pd.io.sql.read_sql(sql=text(query), con = con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           5 non-null      int64 
 1   author_id         5 non-null      int64 
 2   title             5 non-null      object
 3   num_pages         5 non-null      int64 
 4   publication_date  5 non-null      object
 5   publisher_id      5 non-null      int64 
dtypes: int64(4), object(2)
memory usage: 368.0+ bytes


In [8]:
pd.io.sql.read_sql(sql=text(query), con = con).describe()

,book_id,author_id,num_pages,publisher_id
count,5.000000,5.000000,5.000000,5.000000
mean,3.000000,325.000000,567.000000,228.200000
std,1.581139,208.694753,262.114479,108.049526
min,1.000000,82.000000,322.000000,93.000000
25%,2.000000,125.000000,386.000000,135.000000
50%,3.000000,407.000000,541.000000,268.000000
75%,4.000000,465.000000,594.000000,309.000000
max,5.000000,546.000000,992.000000,336.000000


* Посмотрим данные 

In [9]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

In [10]:
con=engine.connect()

In [11]:
pd.io.sql.read_sql(sql=text(query), con = con).describe()

,book_id,author_id,num_pages,publisher_id
count,5.000000,5.000000,5.000000,5.000000
mean,3.000000,325.000000,567.000000,228.200000
std,1.581139,208.694753,262.114479,108.049526
min,1.000000,82.000000,322.000000,93.000000
25%,2.000000,125.000000,386.000000,135.000000
50%,3.000000,407.000000,541.000000,268.000000
75%,4.000000,465.000000,594.000000,309.000000
max,5.000000,546.000000,992.000000,336.000000


In [12]:
query = '''SELECT COUNT(*) FROM books''' # посмтрим количество книг

In [13]:
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,1000


In [14]:
query = '''SELECT COUNT(*) FROM authors''' # кол-во авторов 


In [15]:
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,636


* # **Посчитайте сколько книг вышло после 1 января 2000 года**

In [16]:
query = ''' 
            SELECT COUNT(*)
            FROM books
            WHERE publication_date::date BETWEEN '2000-01-01' and '2222-01-01'
        '''

In [17]:
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,821


* # Для каждой книги посчитайте количество обзоров и среднюю оценку

In [18]:
query = ''' 
            SELECT boo.title,
                   boo.book_id,
                   COUNT(DISTINCT rv.review_id) AS count_rev,
                   ROUND(AVG(r.rating),2) AS avg_rat
            FROM books AS boo
            
            
            LEFT JOIN reviews AS rv
            ON boo.book_id = rv.book_id
            
            LEFT JOIN ratings AS r
            ON boo.book_id = r.book_id
            
            GROUP BY boo.book_id, boo.title
            ORDER BY count_rev DESC, avg_rat DESC
            
        '''

In [19]:
pd.io.sql.read_sql(sql=text(query), con = con)

,title,book_id,count_rev,avg_rat
0,Twilight (Twilight #1),948,7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.41
2,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.29
3,The Book Thief,656,6,4.26
4,The Glass Castle,734,6,4.21
...,...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,191,0,4.00
996,Leonardo's Notebooks,387,0,4.00
997,Essential Tales and Poems,221,0,4.00
998,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.67


* #  Определите издательсво, которое выпустило наибольшее число книг толще 50 страниц, таким образом исключим из анализа брошюры.

In [20]:
query = ''' 
            SELECT publisher, 
                   COUNT(book_id) as books_amount
                   
            FROM books 
            LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY books_amount DESC
            LIMIT 1            
        '''

In [21]:
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_amount
0,Penguin Books,42


* #  Опредилите автора с самой высокой средней оценкой книг - учитывайте только книги с 50 и более оценками

In [22]:
query = '''             
        SELECT author,
               ROUND(AVG(avg_raiting), 3) AS avg_raiting, 
        SUM(rating_count) AS rate_count
        FROM    
        (SELECT author_id, 
                title, 
                AVG(rating) AS avg_raiting, 
                COUNT(rating) AS rating_count
        FROM books 
        LEFT JOIN ratings ON books.book_id = ratings.book_id 
        GROUP BY title, author_id
        HAVING COUNT(rating_id) >= 50) as first
        LEFT JOIN authors ON first.author_id = authors.author_id
        GROUP BY author
        ORDER BY avg_raiting DESC
        LIMIT 1                
        '''

In [23]:
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_raiting,rate_count
0,J.K. Rowling/Mary GrandPré,4.284,310.0


* # Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [24]:
query = '''             
        SELECT ROUND(AVG(rev_counts)) AS avg_rev
        FROM
            (SELECT COUNT(review_id) AS rev_counts
            FROM
                (SELECT username 
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) > 48) AS sub
            LEFT JOIN reviews ON sub.username = reviews.username
            GROUP BY sub.username) AS sub_2
        '''

In [25]:
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_rev
0,24.0


# Вывод

Итоговые выводы
Данные содержат:

1000 книг;
636 авторах;

В результате исследования пришли к следующим выводам:
* 82% книг (809 из 1000) были изданы в 21 веке, а это значит рассчитаны на молодую аудиторию.
Есть над чем задуматся как привлечь аудиторию старше 30+ и получить дополнительную прибыль.
    
* Издательсво Penguin books (42 книги) является рекордсменом по числу книг.
* Самым популярным автором является J.K. Rowling/Mary GrandPré с рейтингом 4.28. 
    
